In [11]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

import ray
ray.init(num_cpus=10)

RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [19]:
# Create Tensors to hold input and outputs.
#1,3,1+1,2,1
beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=[1,2]

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [20]:
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)



    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs_1):
        lambda1,lr,ga= config["lambda1"],config["lr"],config["ga"]
        model_1=MLP(seed=3)
        model_2=MLP(seed=3)
        model_3=MLP(seed=3)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1= model_1(inputs_1[idx_train])
            out_2= model_2(inputs_2[idx_train])
            out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])
           

            model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])
    

            model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            
            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

#             l2_regularization = lambda2* torch.sum(all_linear23_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
#             scheduler.step()
        
        
        test_out_1= model_1(train_x1[idx_test])
        test_out_2= model_2(train_x2[idx_test])
        test_out_3= model_3(train_x3[idx_test])
        
        test_BCE_loss_1 = loss_fn(test_out_1, targets_1[idx_test].reshape(-1,1))
        test_BCE_loss_2 = loss_fn(test_out_2, targets_2[idx_test].reshape(-1,1))
        test_BCE_loss_3 = loss_fn(test_out_3, targets_3[idx_test].reshape(-1,1))
        
        test_BCE_loss=test_BCE_loss_1+test_BCE_loss_2+test_BCE_loss_3
        
        sum_test_BCE_loss+=test_BCE_loss.item()

    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)
     

In [21]:
def train_model_2(config):
    split_num=5
    kf = KFold(n_splits=split_num)
    
    
    lambda1,lr,eta= config["lambda1"], config["lr"],config["eta"]
    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3
    
    
    
    sum_tGM=0
    for idx_train,idx_test in kf.split(inputs_1):
       
        
        model_1=MLP(seed=1)
        model_2=MLP(seed=3)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1 = model_1(inputs_1[idx_train])
            out_2 = model_2(inputs_2[idx_train])
            out_3 = model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])
           

            model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])
    

            model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])
    


            # compute loss

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            
            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))
            
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out_1= model_1(train_x1[idx_test])
        test_out_2= model_2(train_x2[idx_test])
        test_out_3= model_3(train_x3[idx_test])
        
        test_prediction_1= (model_1(train_x1[idx_test])>0.5).clone().int()
        test_target_1= train_y1[idx_test].reshape(-1,1).int()

        test_prediction_2= (model_2(train_x2[idx_test])>0.5).clone().int()
        test_target_2= train_y2[idx_test].reshape(-1,1).int()

        test_prediction_3= (model_3(train_x3[idx_test])>0.5).clone().int()
        test_target_3= train_y3[idx_test].reshape(-1,1).int()

        test_prediction=np.append(test_prediction_1,test_prediction_2)
        test_prediction=np.append(test_prediction,test_prediction_3)
        test_prediction=test_prediction.tolist()

        test_target=np.append(test_target_1,test_target_2)
        test_target=np.append(test_target,test_target_3)
        test_target=test_target.tolist()

        final_matrix=confusion_matrix(test_target,test_prediction)
        tTPR=recall_score(test_target,test_target)#TPR
        tTNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
        tGM=math.sqrt(tTPR*tTNR)

        sum_tGM+=tGM

    tune.report(my_test_sum_tGM=sum_tGM/split_num)
     

In [22]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

(raylet) [2022-11-02 15:30:56,833 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 192770048; capacity: 29394726912. Object creation will fail if spilling is required.


In [23]:
for i in range(10):
    setup_seed(i)
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)

    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    prior=[1,2]

# choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out

    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "ga":tune.choice([0.3,0.4,0.5])
        }

    result = tune.run(
            train_model,  
            config=config,
            num_samples=30)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lr,ga=final_lambda["lambda1"],final_lambda["lr"],final_lambda["ga"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]
    
    model_1=MLP(seed=1)
    model_2=MLP(seed=3)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

    loss_record=[]
    
    for t in range(max_iteration):
    
    # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1= model_1(inputs_1)
        out_2= model_2(inputs_2)
        out_3= model_3(inputs_3)



        model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])


        model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])


        model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)

        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))




    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization 

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()

    
################## Y_prior##################        
    y_prior_1= (model_1(train_x1)).detach().squeeze()

    y_prior_2= (model_2(train_x2)).detach().squeeze()

    y_prior_3= (model_3(train_x3)).detach().squeeze()
    
    
################## third step##################

    #no prior
    prior=[]
    # choose lambda1,lambda2 by ray tune   
        
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out
        
        
    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
         "eta": tune.choice([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]),
         "lr":tune.choice([0.6,0.7,0.8])
                            }

    result = tune.run(
            train_model_2,  
            config=config,
            num_samples=100)
    

################## fourth step##################

    #no prior
    prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    lambda1,eta,lr=final_parameter["lambda1"],final_parameter["eta"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000

    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3

    # record loss descent
    BCE=[]


    # main nn object
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out



    model_1=MLP(seed=3)
    model_2=MLP(seed=3)
    model_3=MLP(seed=3)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

    # learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
    # loop for max_iteration times
    
    for t in range(max_iteration):
    
    # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1= model_1(inputs_1)
        out_2= model_2(inputs_2)
        out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#

        model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])


        model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])


        model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])

        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))
        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
####################################   prediction ####################################      

    prediction_1= (model_1(test_x1)>0.5).clone().int()
    target_1= test_y1.reshape(-1,1).int()

    prediction_2= (model_2(test_x2)>0.5).clone().int()
    target_2= test_y2.reshape(-1,1).int()

    prediction_3= (model_3(test_x3)>0.5).clone().int()
    target_3= test_y3.reshape(-1,1).int()

    prediction=np.append(prediction_1,prediction_2)
    prediction=np.append(prediction,prediction_3)
    prediction=prediction.tolist()

    target=np.append(target_1,target_2)
    target=np.append(target,target_3)
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))
    
    
####################################   variable selection ####################################     
    model_1_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_2_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_3_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=cubic_root(model_1_weight[0,j]*model_2_weight[0,j]*model_3_weight[0,j])

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_1_weight[0,j]=0
            model_2_weight[0,j]=0
            model_3_weight[0,j]=0
        else:
            model_1_weight[0,j]=model_1_weight[0,j]
            model_2_weight[0,j]=model_2_weight[0,j]
            model_3_weight[0,j]=model_3_weight[0,j]
    
    index=np.nonzero(model_1_weight)[1]
    
    vs[:,index]+=1
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    TP=len(trueindex.intersection(index))
    FP=len(np.setdiff1d(index,trueindex))
    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)
    

Trial name,status,loc,ga,lambda1,lr,iter,total time (s),my_test_BCE_loss
train_model_4d80e_00000,TERMINATED,10.224.0.12:25773,0.5,0.03,0.8,1,6.46361,0.689256
train_model_4d80e_00001,TERMINATED,10.224.0.12:25957,0.3,0.05,0.6,1,6.28077,0.7993
train_model_4d80e_00002,TERMINATED,10.224.0.12:25959,0.3,0.05,0.6,1,6.51824,0.7993
train_model_4d80e_00003,TERMINATED,10.224.0.12:25961,0.4,0.03,0.8,1,6.05541,0.689256
train_model_4d80e_00004,TERMINATED,10.224.0.12:25963,0.4,0.05,0.8,1,5.97008,0.799877
train_model_4d80e_00005,TERMINATED,10.224.0.12:25965,0.4,0.04,0.7,1,6.11481,0.740442
train_model_4d80e_00006,TERMINATED,10.224.0.12:25967,0.5,0.04,0.7,1,6.03618,0.740442
train_model_4d80e_00007,TERMINATED,10.224.0.12:25969,0.4,0.04,0.6,1,5.9552,0.739426
train_model_4d80e_00008,TERMINATED,10.224.0.12:25971,0.4,0.04,0.7,1,5.96624,0.740442
train_model_4d80e_00009,TERMINATED,10.224.0.12:25973,0.3,0.04,0.7,1,5.52812,0.740442


(raylet) [2022-11-02 15:31:06,841 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 191340544; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_4d80e_00000:
  date: 2022-11-02_15-31-11
  done: false
  experiment_id: 0b0224f8fa4642a48c28d725507bf917
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6892563700675964
  node_ip: 10.224.0.12
  pid: 25773
  time_since_restore: 6.463611125946045
  time_this_iter_s: 6.463611125946045
  time_total_s: 6.463611125946045
  timestamp: 1667374271
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 4d80e_00000
  warmup_time: 0.0027549266815185547
  
Result for train_model_4d80e_00000:
  date: 2022-11-02_15-31-11
  done: true
  experiment_id: 0b0224f8fa4642a48c28d725507bf917
  experiment_tag: 0_ga=0.5000,lambda1=0.0300,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6892563700675964
  node_ip: 10.224.0.12
  pid: 25773
  time_since_restore: 6.463611125946045
  time_this_iter_s: 6.463611125946045
  time_total_s: 6.463611125946045
  timestamp: 1667374271
  timesteps_since_restore: 

Result for train_model_4d80e_00002:
  date: 2022-11-02_15-31-14
  done: false
  experiment_id: fec1b8c0595349c8a2a632cd51813058
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7993002653121948
  node_ip: 10.224.0.12
  pid: 25959
  time_since_restore: 6.5182414054870605
  time_this_iter_s: 6.5182414054870605
  time_total_s: 6.5182414054870605
  timestamp: 1667374274
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 4d80e_00002
  warmup_time: 0.0035092830657958984
  
Result for train_model_4d80e_00002:
  date: 2022-11-02_15-31-14
  done: true
  experiment_id: fec1b8c0595349c8a2a632cd51813058
  experiment_tag: 2_ga=0.3000,lambda1=0.0500,lr=0.6000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7993002653121948
  node_ip: 10.224.0.12
  pid: 25959
  time_since_restore: 6.5182414054870605
  time_this_iter_s: 6.5182414054870605
  time_total_s: 6.5182414054870605
  timestamp: 1667374274
  timesteps_since_res

(raylet) [2022-11-02 15:31:16,845 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 190128128; capacity: 29394726912. Object creation will fail if spilling is required.


...

(raylet) [2022-11-02 16:35:39,559 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 24875008; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:35:49,568 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 24788992; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:35:59,576 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 24760320; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:36:09,585 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 24690688; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-02 16:40:49,818 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 23068672; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:40:59,827 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 23019520; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:41:09,836 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 22962176; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:41:19,844 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 22941696; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-02 16:46:00,077 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 21340160; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:46:10,086 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 21295104; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:46:20,094 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 21270528; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:46:30,103 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 21155840; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-02 16:51:10,337 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 17973248; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:51:20,352 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 16965632; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:51:30,368 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 15949824; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:51:40,376 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 15192064; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-02 16:56:20,661 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 4857856; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:56:30,669 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 4771840; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:56:40,678 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 4751360; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:56:50,686 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 4636672; capacity: 29394726912. Object creation will fail if spilli

In [17]:
####################################   prediction ####################################   
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)


0.9078243599867234

0.9257669047333771

0.9161919790266702

0.9166666666666667

0.9252551474506305

0.9153913617123054

(raylet) [2022-11-02 15:29:46,804 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 202448896; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:29:56,808 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 201322496; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:30:06,813 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 199868416; capacity: 29394726912. Object creation will fail if spilling is required.


In [18]:
####################################   variable selection ####################################   
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

1.0

0.8866666666666667

0.9416268242330533

0.10200000000000001

0.898

(raylet) [2022-11-02 15:30:16,818 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 198303744; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:30:26,822 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 197120000; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:30:36,826 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 195727360; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:30:46,830 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 194248704; capacity: 29394726912. Object creation will fail i

In [10]:
vs

array([[10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

(raylet) [2022-11-02 14:35:53,895 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 376352768; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:36:03,904 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 376336384; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:36:13,913 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 376283136; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:36:23,922 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 376250368; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-02 14:41:04,181 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 375074816; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:41:14,190 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 375021568; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:41:24,199 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 374996992; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:41:34,208 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 374960128; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-02 14:46:14,478 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 373760000; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:46:24,487 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 373731328; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:46:34,497 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 373686272; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:46:44,505 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 373608448; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-02 14:51:24,762 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 372469760; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:51:34,772 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 372432896; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:51:44,781 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 372350976; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:51:54,790 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 372326400; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-02 14:56:35,052 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 371163136; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:56:45,061 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 371085312; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:56:55,070 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 371052544; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:57:05,078 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 371027968; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-02 15:01:45,331 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 369803264; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:01:55,341 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 369790976; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:02:05,350 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 369770496; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:02:15,358 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 369721344; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-02 15:06:55,614 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 368533504; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:07:05,623 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 368513024; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:07:15,631 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 368459776; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:07:25,640 E 17271 17289] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_14-14-41_060076_17160 is over 95% full, available space: 368431104; capacity: 29394726912. Object creation will fail i

In [10]:
i

9

(raylet) [2022-11-02 14:10:05,243 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 441094144; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:10:15,252 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 441049088; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:10:25,262 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 441028608; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 14:10:35,271 E 1818 1845] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_11-09-23_361185_1341 is over 95% full, available space: 440979456; capacity: 29394726912. Object creation will fail if spilling i